In [45]:
import open3d as o3d
import numpy as np

# --- Load & normalize (mm -> m, center at origin) ---
mesh = o3d.io.read_triangle_mesh("Bolt/hex_bolt_30.stl")
mesh.compute_vertex_normals()
mesh.scale(0.001, center=mesh.get_center())     # mm -> m (adjust if already meters)
mesh.translate(-mesh.get_center())              # origin at centroid

# --- Dense surface sampling ---
pcd = mesh.sample_points_poisson_disk(20000)    # increase if you want more detail
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

# --- Model resolution (avg NN distance) ---
dists = pcd.compute_nearest_neighbor_distance()
res = float(np.mean(dists))

# --- ISS keypoints (radii set relative to resolution) ---
iss = o3d.geometry.keypoint.compute_iss_keypoints(
    pcd,
    salient_radius=6 * res,     # try 6–8 * res
    non_max_radius=4 * res,     # try 4–6 * res
    gamma_21=0.975,
    gamma_32=0.975,
    min_neighbors=10
)

# Optional: thin out duplicates if very dense
iss = iss.voxel_down_sample(voxel_size=2 * res)

print(f"ISS keypoints: {np.asarray(iss.points).shape[0]}")

# Visualize (gray = cloud, red = keypoints)
pcd.paint_uniform_color([0.7, 0.7, 0.7])
iss.paint_uniform_color([1.0, 0.0, 0.0])
o3d.visualization.draw_geometries([pcd, iss])


ISS keypoints: 196


In [ ]:
import numpy as np
import open3d as o3d
from collections import defaultdict

mesh = o3d.io.read_triangle_mesh("Bolt/hex_bolt_30.stl")
mesh.compute_triangle_normals()
mesh.compute_vertex_normals()
mesh.scale(0.001, center=mesh.get_center())
mesh.translate(-mesh.get_center())

V = np.asarray(mesh.vertices)
T = np.asarray(mesh.triangles)
N = np.asarray(mesh.triangle_normals)

# Build edge -> triangle adjacency
edge_to_tris = defaultdict(list)
def ekey(a, b):
    return (a, b) if a < b else (b, a)

for ti, (a, b, c) in enumerate(T):
    edge_to_tris[ekey(a,b)].append(ti)
    edge_to_tris[ekey(b,c)].append(ti)
    edge_to_tris[ekey(c,a)].append(ti)

# Compute dihedral angles and collect "sharp" edges
sharp_edges = []
deg_thresh = 30.0
cos_thresh = np.cos(np.deg2rad(180 - deg_thresh))  # compare angle between face normals

for (i, j), tris in edge_to_tris.items():
    if len(tris) != 2:
        continue  # boundary/non-manifold: skip or keep as sharp
    n1, n2 = N[tris[0]], N[tris[1]]
    cosang = np.clip(np.dot(n1, n2), -1.0, 1.0)
    angle = np.arccos(cosang)  # radians, 0 = coplanar, pi = folded
    dihedral = np.pi - angle
    if np.degrees(dihedral) > deg_thresh:
        sharp_edges.append((i, j))

# Vertices incident to many sharp edges -> corners
sharp_count = np.zeros(len(V), dtype=int)
for i, j in sharp_edges:
    sharp_count[i] += 1
    sharp_count[j] += 1

corner_idx = np.where(sharp_count >= 3)[0]   # tune: 2 or 3 for hex bolt heads
corner_pts = V[corner_idx]

# Make a point cloud for visualization
corner_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(corner_pts))
corner_pcd.paint_uniform_color([1, 0, 0])

mesh.paint_uniform_color([0.7, 0.7, 0.7])
o3d.visualization.draw_geometries([mesh, corner_pcd])
print(f"Detected corner vertices: {len(corner_pts)}")


Detected corner vertices: 32


In [47]:
# Combine sets, then deduplicate with voxel grid
combined = o3d.geometry.PointCloud()
combined += iss
combined += o3d.geometry.PointCloud(o3d.utility.Vector3dVector(corner_pts))

combined = combined.voxel_down_sample(voxel_size=2 * res)  # unify close points
combined.paint_uniform_color([1, 0, 0])

K3D = np.asarray(combined.points)   # final 3D keypoints